<a href="https://colab.research.google.com/github/WillNovus/Tasks/blob/main/Facial_Detection_and_Recognition_with_VGGFace2_for_KYC_protocol_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
!pip install mtcnn

In [ ]:
!pip show keras-vggface

In [ ]:
import mtcnn

In [ ]:
 !pip install keras-Applications

In [ ]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os 
root_dir = "/content/drive/My Drive/"
project_folder = "Colab Notebooks/project_folder/"
def create_and_set_working_directory(project_folder):
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')
  os.chdir(root_dir + project_folder)
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )
  
create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab Notebooks/project_folder/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/project_folder


In [ ]:
def extract_face(filename, required_size=(224, 224)):
  pixels = pyplot.imread(filename)
  # detect faces in the image
  detector = MTCNN()
  results = detector.detect_faces(pixels)
  # extract the bounding box from the first face
  x1, y1, width, height = results[0]['box']
  x2, y2 = x1 + width, y1 + height
  # extract the face
  face = pixels[y1:y2, x1:x2]
  # resize pixels to the model size
  image = Image.fromarray(face)
  image = image.resize((224, 224))
  face_array = asarray(image)
  return face_array

  


In [ ]:
extract_face('boss11.jpg')

array([[[220, 225, 216],
        [221, 225, 216],
        [221, 226, 216],
        ...,
        [211, 218, 211],
        [211, 218, 210],
        [211, 217, 210]],

       [[219, 222, 213],
        [220, 223, 214],
        [218, 222, 213],
        ...,
        [213, 220, 213],
        [212, 219, 211],
        [212, 218, 211]],

       [[219, 222, 213],
        [219, 222, 213],
        [220, 223, 214],
        ...,
        [212, 218, 211],
        [212, 218, 211],
        [212, 217, 210]],

       ...,

       [[126, 131, 141],
        [124, 127, 141],
        [120, 123, 140],
        ...,
        [102, 109, 127],
        [ 94, 102, 120],
        [130, 135, 145]],

       [[ 91,  97, 119],
        [ 92,  97, 119],
        [ 95, 101, 120],
        ...,
        [ 95, 103, 120],
        [ 94, 104, 121],
        [ 99, 107, 125]],

       [[ 90,  97, 116],
        [ 77,  85, 103],
        [ 78,  84, 100],
        ...,
        [ 89,  99, 114],
        [ 91, 102, 121],
        [ 93, 104, 124]]

In [ ]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
# extract faces
  faces = [extract_face(f) for f in filenames]
# convert into an array of samples
  samples = asarray(faces, 'float32')
# prepare the face for the model, e.g. center pixels. I believe this step solves your resolution problem. 
  samples = preprocess_input(samples, version=2)
# create a vggface model
  model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),
  pooling='avg')
# perform prediction
  yhat = model.predict(samples)
  return yhat

In [ ]:
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
# calculate distance between embeddings
  score = cosine(known_embedding, candidate_embedding)
  if score <= thresh:
    print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
  else:
    print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))


In [ ]:
# define filenames
filenames = ['boss11.jpg', 'boss12.jpg', 'lily15.jpg','lily17.jpg',
'lily18.jpg', 'paul12.jpg', 'paul17.jpg', 'sam13.jpg','sam14.jpg' ]
# get embeddings file filenames
embeddings = get_embeddings(filenames)


In [ ]:

# define Mario, Lily, Paul, Sam
Mario_id = embeddings[0]
Lily_id = embeddings[2]
Paul_id = embeddings[5]
Sam_id = embeddings[6]
# verify known photos of Mario
print('Match Test1')
is_match(embeddings[0], embeddings[1])
# verify known photos of Lily
print('Match Test2')
is_match(embeddings[7], embeddings[8])

Match Test1
>face is a Match (0.399 <= 0.500)
Match Test2
>face is a Match (0.111 <= 0.500)


In [ ]:
# For Negative Result
print ('Match Test3')
is_match(embeddings[5], embeddings[6])

Match Test3
>face is NOT a Match (0.660 > 0.500)
